# Griegas / Griegas Numericas

In [1]:
import QuantLib as ql

## Griegas Analiticas

In [2]:

#Definimos los inputs
precio_activo = 440 #Spot (S)
precio_ejercicio = 450 #Strike (K)
volatilidad = 0.3394  #Volatilidad Implicita anualizada (sigma)
tasa_interes = 0.0013 #tasa libre de riesgo anualizada (r)
tasa_dividendos =  0.075 #tasa de dividendos anualizada (q)

#Expiracion el 16 de octubre de 2020
fecha_expiracion = ql.Date(16, 10, 2020)
#Valuacion el 16 de octubre de 2020
fecha_valuacion = ql.Date(6, 8, 2020)
ql.Settings.instance().evaluationDate = fecha_valuacion
day_count = ql.Actual365Fixed()
calendario = ql.UnitedStates()


In [3]:
tipo_opcion = ql.Option.Call #Tipo de opcion (CALL o PUT)
payoff = ql.PlainVanillaPayoff(tipo_opcion, precio_ejercicio)
ejercicio_europeo = ql.EuropeanExercise(fecha_expiracion)
opcion_europea = ql.VanillaOption(payoff, ejercicio_europeo)

In [4]:
S = ql.SimpleQuote(precio_activo)
objeto_spot = ql.QuoteHandle(S)

r = ql.SimpleQuote(tasa_interes)
objeto_tasa_interes = ql.YieldTermStructureHandle(ql.FlatForward(fecha_valuacion, 
                                                            ql.QuoteHandle(r), 
                                                        day_count))
q = ql.SimpleQuote(tasa_dividendos)
objeto_tasa_dividendos = ql.YieldTermStructureHandle(ql.FlatForward(fecha_valuacion, 
                                                      ql.QuoteHandle(q), 
                                                      day_count))

sigma = ql.SimpleQuote(volatilidad)
objeto_volatilidad = ql.BlackVolTermStructureHandle(ql.BlackConstantVol(fecha_valuacion, 
                                                                 calendario, 
                                                                 ql.QuoteHandle(sigma), 
                                                             day_count))

proceso_BSM = ql.BlackScholesMertonProcess(objeto_spot, 
                                           objeto_tasa_dividendos, 
                                           objeto_tasa_interes, 
                                           objeto_volatilidad)

opcion_europea.setPricingEngine(ql.AnalyticEuropeanEngine(proceso_BSM))

En el caso de la formula Analitica, QuantLib ofrece la posibilidad de extraer varias sensibilidades (griegas) de la opcion. Entre ellos: `delta()`,  `vega()`, `rho() `, `dividendRho()`, `theta()`, `thetaPerDay()`, `strikeSensitivity()`, `gamma()`, `elasticity()`.


In [5]:
delta_BS = opcion_europea.delta()
vega_BS = opcion_europea.vega()
rho_BS = opcion_europea.rho() 
div_rho_BS = opcion_europea.dividendRho()
theta_BS = opcion_europea.theta()
thetaPD_BS = opcion_europea.thetaPerDay()
strike_greek_BS = opcion_europea.strikeSensitivity()

gamma_BS = opcion_europea.gamma()
elasticity_BS = opcion_europea.elasticity()


print("Delta: ", delta_BS)
print("Vega:", vega_BS)
print("Rho: ", rho_BS)
print("dividendRho: ", div_rho_BS)
print("Theta: ", theta_BS)
print("Theta_PD: ", thetaPD_BS)
print("strikeSensitivity: ", strike_greek_BS)

print("Gamma: ", gamma_BS)
print("Elasticity", elasticity_BS)





Delta:  0.4258321048466277
Vega: 75.18926359141635
Rho:  32.74716062566774
dividendRho:  -36.446561521667505
Theta:  -51.76161250063861
Theta_PD:  -0.141812636988051
strikeSensitivity:  -0.374106842828442
Gamma:  0.0058826519800777724
Elasticity 9.852017271520342


## Griegas Numericamente

Si llamamos $V$ al derivado en cuestion, y el spot vale $S$ a tiempo inicial, entonces la Delta y la Gamma no son otra cosa que:

$$
\Delta = \frac{V(S+h)-V(S-h)}{2h} \; \; \; \; 
\Gamma = \frac{V(S+h)-2V(S)+V(S-h)}{h^2}
$$

con $h$ un numero sificientemente chico (en general se suele tomar la unidad indivisible en cuestion) y suponemos que todos los demas parametros quedan fijo

Gracias al framwork que nos provee QuantLib, buscar los precios perturbados es sencillo. Necesitamos simplemente cambiar el parametro en cuestion y volver a calcular:

In [6]:
tipo_opcion = ql.Option.Call #Tipo de opcion (CALL o PUT)
payoff = ql.PlainVanillaPayoff(tipo_opcion, precio_ejercicio)
ejercicio_europeo = ql.EuropeanExercise(fecha_expiracion)
opcion_europea = ql.VanillaOption(payoff, ejercicio_europeo)


S = ql.SimpleQuote(precio_activo)
objeto_spot = ql.QuoteHandle(S)

r = ql.SimpleQuote(tasa_interes)
objeto_tasa_interes = ql.YieldTermStructureHandle(ql.FlatForward(fecha_valuacion, 
                                                            ql.QuoteHandle(r), 
                                                        day_count))
q = ql.SimpleQuote(tasa_dividendos)
objeto_tasa_dividendos = ql.YieldTermStructureHandle(ql.FlatForward(fecha_valuacion, 
                                                      ql.QuoteHandle(q), 
                                                      day_count))

sigma = ql.SimpleQuote(volatilidad)
objeto_volatilidad = ql.BlackVolTermStructureHandle(ql.BlackConstantVol(fecha_valuacion, 
                                                                 calendario, 
                                                                 ql.QuoteHandle(sigma), 
                                                             day_count))

proceso_BSM = ql.BlackScholesMertonProcess(objeto_spot, 
                                           objeto_tasa_dividendos, 
                                           objeto_tasa_interes, 
                                           objeto_volatilidad)
modelo_BS = ql.AnalyticEuropeanEngine(proceso_BSM)

opcion_europea.setPricingEngine(modelo_BS)



In [7]:
S0 = S.value()
S.setValue(S0)
h = 0.001 #(1 centavo de USD)
V0 = opcion_europea.NPV() 
print(V0)

19.018046859717114


Incrementamos el precio del activo y obtenemos el precio de la opcion (plus)

In [8]:
S.setValue(S0+h)
V_p = opcion_europea.NPV()
print(V_p)

19.018472694763293


Lo mismo restando y obtenemos el precio de la opcion (minus)

In [9]:
S.setValue(S0-h)
V_m = opcion_europea.NPV()
print(V_m)

19.017621030553624


Finalmente, volvemos a dejar a S con el precio original

In [10]:
S.setValue(S0)

Aplico las formulas anteriores para calcular las griegas numericas:

In [11]:
delta_numerica = (V_p - V_m)/(2*h)
gamma_numerica = (V_p - 2*V0 + V_m)/(h*h)

print("Delta numerica:", delta_numerica)
print("Delta BS:", delta_BS)

print("Gamma numerica:", gamma_numerica)
print("Gamma BS:", gamma_BS)

                  
                  

Delta numerica: 0.42583210483471134
Delta BS: 0.4258321048466277
Gamma numerica: 0.0058826898907682335
Gamma BS: 0.0058826519800777724


La misma idea es para las demas griegas. Por ejemplo:

$$Vega = \frac{V(\sigma+h)-V(\sigma-h)}{2h} \; \; \; \; 
\rho = \frac{V(r+h)-V(r-h)}{2h}
$$


In [12]:
sigma0 = sigma.value() 
h = 0.0001

sigma.setValue(sigma0+h) 
V_plus = opcion_europea.NPV()

sigma.setValue(sigma0)

vega_numerica = (V_plus - V0)/h 


print("Vega numerica:", vega_numerica)
print("Vega BS:", vega_BS)


Vega numerica: 75.18987112291597
Vega BS: 75.18926359141635


In [13]:
r0 = r.value()
h = 0.0001
r.setValue(r0+h)
V_plus = opcion_europea.NPV()
r.setValue(r0)
rho_numerica = (V_plus - V0)/h


print("Rho tasa numerica:", rho_numerica)
print("Rho tasa BS:", rho_BS)

Rho tasa numerica: 32.74899679407639
Rho tasa BS: 32.74716062566774


In [14]:
q0 = q.value()
h = 0.0001
q.setValue(q0+h)
V_plus = opcion_europea.NPV()
q.setValue(q0)
div_rho_numerica = (V_plus - V0)/h


print("Rho dividendo numerica:", div_rho_numerica)
print("Rho dividendo BS:", div_rho_BS)

Rho dividendo numerica: -36.44405242358317
Rho dividendo BS: -36.446561521667505


El enfoque para Theta es un poco diferente, aunque todavía se basa en el hecho de que la opción reacciona al cambio en los datos del mercado. El problema es que no tenemos el tiempo de vencimiento disponible como cotización, como sucedía con las otras cantidades. En cambio, dado que configuramos las estructuras de términos para que se muevan con la fecha de evaluación, solo tenemos que configurarlo en la fecha de mañana para obtener el valor de opción correspondiente:

In [15]:
##No esta recalculando con la nueva fecha!


ql.Settings.instance().evaluationDate = fecha_valuacion +10


V1 = opcion_europea.NPV()

h = 1.0/365.0
print(V1)
print(V0)
theta_numerica = (V1-V0)/h





print("Theta numerica:", theta_numerica)
print("Theta BS:", theta_BS)

19.018046859717114
19.018046859717114
Theta numerica: 0.0
Theta BS: -51.76161250063861


In [16]:
#vuelvo
ql.Settings.instance().evaluationDate = fecha_valuacion